In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split

In [4]:
n_images: int = 50_000
n_noisy: int = 40_000
n_clean: int = n_images - n_noisy

images : np.ndarray = np.empty((n_images, 32, 32, 3), dtype=np.float32)

# Load the data
for i in range(n_images):
    image_path = f"../data/images/{i+1:05d}.png"
    images[i,:,:,:] = cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB)

# load the labels
clean_labels = np.genfromtxt('../data/clean_labels.csv', delimiter=',', dtype="int8")
noisy_labels = np.genfromtxt('../data/noisy_labels.csv', delimiter=',', dtype="int8")

In [7]:
cnn = ResNet18(10)
cnn.build(input_shape = (None,32,32,3))

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [1]:
cnn.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = "adam",
    metrics = ["accuracy"]
)

NameError: name 'cnn' is not defined

In [14]:
x_full = tf.cast(images, dtype = tf.float32) / 255.0
y_clean = tf.one_hot(clean_labels, depth = 10)
y_noisy = tf.one_hot(noisy_labels, depth = 10)

In [15]:
x_clean = x_full[:n_clean]

In [18]:
aug = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, width_shift_range=0.05,
                             height_shift_range=0.05)
aug.fit(x_clean)

In [20]:
es = tf.keras.callbacks.EarlyStopping(patience= 8, restore_best_weights=True, monitor="val_acc")

In [22]:
STEPS = len(x_clean) / 256
cnn.fit(
    aug.flow(x_clean, y_clean, batch_size = 256),
    steps_per_epoch=STEPS,
    batch_size = 256,
    epochs=5,
    validation_data = (x_clean, y_clean),
    callbacks=[es]
)

Epoch 1/5
 6/39 [===>..........................] - ETA: 4:45 - loss: 2.8367 - accuracy: 0.1992

KeyboardInterrupt: 